# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Experiment, Dataset
from azureml.core.dataset import Dataset
import logging
from azureml.train.automl import AutoMLConfig

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
data_path = "https://raw.githubusercontent.com/jaircastruita/nd00333-capstone/master/starter_file/data/WA_Fn-UseC_-HR-Employee-Attrition.csv"

ibm_ds = Dataset.Tabular.from_delimited_files(path=data_path)

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-experiment'

experiment=Experiment(ws, experiment_name)

## Creating a compute instance

In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

cluster_name = "cpu_cluster"

try:
    compute_target = ComputeTarget(ws, cluster_name)
    print("Reusing already created cluster: {}".format(cluster_name))

except ComputeTargetException:
    cluster_config = AmlCompute.provisioning_configuration(vm_size=, max_nodes=6)
    compute_target = ComputeTarget.create(workspace=ws, cluster_name, cluster_config)

compute_target.wait_for_completion(show_output=True)

## AutoML Configuration

TODO: Explain why you chose the automl settings and configuration you used below.

In [ ]:
# AUC_weighted
# TODO: Put your automl settings here
automl_settings = {"iteration_timeout_minutes": 20,
                    "enable_early_stopping": True,
                    "primary_metric": "accuracy",
                    "verbosity": logging.INFO,
                    "max_concurrent_iterations": 5,
                    "n_cross_validation": 5
                    }

# TODO: Put your automl config here
automl_config = AutoMLConfig(task="classification",
                             compute_target=compute_target,
                             training_data=ibm_ds,
                             label_column_name="Attrition",
                             **automl_settings)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_automl_run, best_automl_model = remote_run.get_output()

In [ ]:
#TODO: Save the best model
best_run.automl_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/automl_score.py')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service